# Read SpaceX Launch Data from https://api.spacexdata.com/v4

Documentation of the SpaceX API can be found @ https://github.com/r-spacex/SpaceX-API/tree/master/docs

Personal GitLab @ https://github.com/gilberpl/data-engineeering/blob/main/analysis.ipynb

In [1]:
import requests

# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [2]:
print(f"There are {len(launches)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

There are 205 launches
There are 225 payloads
There are 4 rockets


In [3]:
import os
import sqlite3

# Delete the old database file if it exists
if os.path.exists("spacex.db"):
    os.remove("spacex.db")
    print("spacex.db database deleted")

spacex.db database deleted


In [4]:
# Create a connection (this creates spacex.db in your current Colab environment)
conn = sqlite3.connect("spacex.db")

# Create a cursor
cur = conn.cursor()

schema = """
CREATE TABLE IF NOT EXISTS rockets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    cost_per_launch INTEGER

);

CREATE TABLE IF NOT EXISTS launches (
    id TEXT PRIMARY KEY,
    name TEXT,
    date_utc TEXT,
    rocket_id TEXT
);

CREATE TABLE IF NOT EXISTS payloads (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    mass_kg INTEGER,
    launch_id TEXT
);
"""

cur.executescript(schema)
conn.commit()
print("Database schema created")

Database schema created


In [5]:
# Insert data from api into rockets table, with only the columns we want
# ignore preventing pk conflicts
for r in rockets:
    cur.execute("""
        INSERT OR IGNORE INTO rockets
        (id, name, type, cost_per_launch)
        VALUES (?, ?, ?, ?)
    """, (
        r["id"], r["name"], r["type"], r["cost_per_launch"]
    ))

conn.commit()
print("Rockets data inserted")

Rockets data inserted


In [6]:
# Insert data from api into launches table, with only the columns we want
# ignore preventing pk conflicts
for l in launches:
    cur.execute("""
        INSERT OR IGNORE INTO launches
        (id, name, date_utc, rocket_id)
        VALUES (?, ?, ?, ?)
    """, (
        l["id"], l["name"], l["date_utc"], l["rocket"]
    ))

conn.commit()
print("Launches data inserted")

Launches data inserted


In [7]:
# Insert data from api into payloads table, with only the columns we want
# ignore preventing pk conflicts
for p in payloads:
    cur.execute("""
        INSERT OR IGNORE INTO payloads
        (id, name, type, mass_kg, launch_id)
        VALUES (?, ?, ?, ?, ?)
    """, (
        p["id"], p["name"], p["type"], p["mass_kg"], p["launch"]
    ))

conn.commit()
print("Payloads data inserted")

Payloads data inserted


In [9]:
import pandas as pd
# Which rocket has cost spacex the most to launch over its lifetime
df_1 = pd.read_sql_query('''
WITH launch_count AS (
  SELECT count(*) as num_launch, rocket_id
  FROM launches
  GROUP BY rocket_id
)

SELECT r.name, (lc.num_launch*r.cost_per_launch) as total_launches_cost
FROM launch_count lc
LEFT JOIN rockets r
ON lc.rocket_id = r.id
ORDER BY total_launches_cost DESC
''', conn)
df_1

,name,total_launches_cost
0,Falcon 9,9750000000
1,Falcon Heavy,450000000
2,Falcon 1,33500000


In [10]:
# Average payload weight per launch type broken down by rocket name
df_2 = pd.read_sql_query('''
SELECT p.type, avg(p.mass_kg) as avg_weight, r.name
FROM payloads p
JOIN launches l ON p.launch_id = l.id
JOIN rockets r ON l.rocket_id = r.id
GROUP BY p.type, r.name
ORDER BY r.name DESC
''', conn)
df_2

,type,avg_weight,name
0,Satellite,730.947368,Falcon Heavy
1,Crew Dragon,11434.333333,Falcon 9
2,Dragon 1.0,500.500000,Falcon 9
3,Dragon 1.1,2578.761111,Falcon 9
4,Dragon 2.0,3150.000000,Falcon 9
5,Dragon Boilerplate,NaN,Falcon 9
6,Lander,585.000000,Falcon 9
7,Satellite,8514.844014,Falcon 9
8,Satellite,128.333333,Falcon 1


In [11]:
# Load tables
rockets_df = pd.read_sql_query("SELECT * FROM rockets", conn)
launches_df = pd.read_sql_query("SELECT * FROM launches", conn)
payloads_df = pd.read_sql_query("SELECT * FROM payloads", conn)


# Merge payloads, launches, rockets into one df
df = payloads_df.merge(launches_df, left_on='launch_id', right_on='id', suffixes=('_payload', '_launch'))
df = df.merge(rockets_df, left_on='rocket_id', right_on='id', suffixes=('', '_rocket'))

In [12]:
# Show number of launches per year using Pandas
df['date_utc'] = pd.to_datetime(df['date_utc'])
launches_per_year = df.groupby(df['date_utc'].dt.year)['launch_id'].nunique().reset_index()
launches_per_year = launches_per_year.rename(columns={'launch_id': 'num_launches'})
launches_per_year

,date_utc,num_launches
0,2006,1
1,2007,1
2,2008,2
3,2009,1
4,2010,2
5,2012,2
6,2013,3
7,2014,6
8,2015,7
9,2016,9
